<a href="https://colab.research.google.com/github/NicKylis/letter_recognition/blob/layman/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
#import kagglehub
import shutil
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, ReLU, MaxPooling2D, UpSampling2D, Dropout, BatchNormalization, Flatten, Dense
from cv2 import cv2
from tensorflow.keras.callbacks import ReduceLROnPlateau

print ("hello world!")

hello world!


In [48]:
# def clean_image(img): Unused, increased losses in data
#     kernel = np.ones((3, 3), np.uint8)
#     img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
#     img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
#     return img
def get_MNIST_dataset(train_generator, batch_size=32, val_split=0.2, random_state=42):
    mnist = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0 #Normalize data
    x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0
    #Split data to train and valitation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = val_split, random_state=random_state)

    num_classes = 10  # MNIST has 10 classes (digits 0-9)
    y_train = to_categorical(y_train, num_classes)
    y_val = to_categorical(y_val, num_classes)
    y_test = to_categorical(y_test, num_classes)

    #Use default ImageDataGenerator for inmutable images
    val_test_generator = ImageDataGenerator()
    train_gen = train_generator.flow(x_train, y_train, batch_size=batch_size)
    val_gen = val_test_generator.flow(x_val, y_val, batch_size=batch_size)
    test_gen = val_test_generator.flow(x_test, y_test, batch_size=batch_size, shuffle=False)

    return train_gen, val_gen, test_gen

In [49]:
train_generator = ImageDataGenerator()
train_generator_aug = ImageDataGenerator(#Use this one, higher accuracy percentage
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.2,
    zoom_range=0.2,)

train_data, val_data, test_data = get_MNIST_dataset(train_generator, batch_size=32, val_split=0.2, random_state=42)
train_data_aug, val_data_aug, test_data_aug = get_MNIST_dataset(train_generator, batch_size=32, val_split=0.2, random_state=42)

In [50]:
model = keras.Sequential()
model.add(Conv2D(32, (3, 3), strides=(1, 1), activation=None, input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2)) #Original image feature extraction

#Feature extraction from simulating Opening operation on image
model.add(MaxPooling2D((2, 2)))#Erosion
model.add(UpSampling2D(size=(2, 2)))#Dilation
model.add(Conv2D(64, (3, 3), activation=None))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dropout(0.25))#0.15

#Feature extraction from simulating Closing operation on image
model.add(UpSampling2D(size=(2, 2)))#Dilation
model.add(MaxPooling2D(2, 2))#Erosion
model.add(Conv2D(64, (3, 3), activation=None))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3), activation=None))
model.add(BatchNormalization())
model.add(ReLU())

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))          

from tensorflow.keras.losses import CategoricalCrossentropy
loss_fn = CategoricalCrossentropy(label_smoothing=0.1)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.summary()
#Can use more epochs, should improve performance marginally

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_36 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 batch_normalization_36 (Bat  (None, 26, 26, 32)       128       
 chNormalization)                                                
                                                                 
 re_lu_36 (ReLU)             (None, 26, 26, 32)        0         
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 13, 13, 32)       0         
 g2D)                                                            
                                                                 
 dropout_40 (Dropout)        (None, 13, 13, 32)        0         
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 6, 6, 32)       

In [45]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-6)
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
history = model.fit(train_data_aug,
         validation_data=val_data_aug,
         epochs=40,
         batch_size=32,
         shuffle=True
         ,callbacks=[lr_scheduler])

Epoch 1/40
1500/1500 [==============================] - 14s 9ms/step - loss: 0.6923 - accuracy: 0.9504 - val_loss: 0.5867 - val_accuracy: 0.9768 - lr: 0.0010
Epoch 2/40
1500/1500 [==============================] - 14s 9ms/step - loss: 0.5990 - accuracy: 0.9819 - val_loss: 0.5639 - val_accuracy: 0.9872 - lr: 0.0010
Epoch 3/40
1500/1500 [==============================] - 14s 9ms/step - loss: 0.5837 - accuracy: 0.9856 - val_loss: 0.5512 - val_accuracy: 0.9876 - lr: 0.0010
Epoch 4/40
1500/1500 [==============================] - 14s 9ms/step - loss: 0.5742 - accuracy: 0.9877 - val_loss: 0.5488 - val_accuracy: 0.9892 - lr: 0.0010
Epoch 5/40
1500/1500 [==============================] - 14s 9ms/step - loss: 0.5665 - accuracy: 0.9898 - val_loss: 0.5378 - val_accuracy: 0.9905 - lr: 0.0010
Epoch 6/40
1500/1500 [==============================] - 14s 9ms/step - loss: 0.5637 - accuracy: 0.9903 - val_loss: 0.5404 - val_accuracy: 0.9898 - lr: 0.0010
Epoch 7/40
1500/1500 [==============================

In [46]:
loss, acc = model.evaluate(test_data_aug)
print(f'The accuracy of the model is {round(acc * 100, 2)}%')
print(f'The loss of the model is {round(loss, 4)}')

313/313 [==============================] - 2s 5ms/step - loss: 0.5178 - accuracy: 0.9952
The accuracy of the model is 99.52%
The loss of the model is 0.5178


In [51]:
model.save('best_weights.h5')